<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [29]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [30]:
import folium
import pandas as pd

In [31]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [ ]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [32]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [33]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [35]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [36]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [38]:
# Loop through the launch sites in the launch_sites_df DataFrame
for index, site in launch_sites_df.iterrows():
    # Extract the coordinate
    coordinate = [site['Lat'], site['Long']]
    
    # Create a circle for each launch site
    circle = folium.Circle(
        coordinate, 
        radius=1000, 
        color='#000000', 
        fill=True
    ).add_child(folium.Popup(site['Launch Site']))
    
    # Create a marker for each launch site
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site["Launch Site"]}</b></div>'
        )
    )
    
    # Add the circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


<div style="text-align: center; font-weight: bold;">
    Proximity to the Equator
</div>

Launch sites like Kennedy Space Center (28.5721° N, 80.6480° W) and Cape Canaveral (28.5623° N, 80.5774° W) are relatively close to the Equator compared to other sites like Vandenberg (34.6328° N, 120.6107° W). While these sites are not exactly on the Equator, they are positioned to take advantage of the Earth's rotational speed, which is beneficial for launching spacecraft into orbit.

<div style="text-align: center; font-weight: bold;">
    Proximity to the Coast
</div>

Most launch sites are indeed very close to the coast. For example, Kennedy Space Center and Cape Canaveral are located on the coast of Florida, allowing rockets to be launched over the Atlantic Ocean. Vandenberg Air Force Base is also near the coast of California, allowing launches over the Pacific Ocean. This proximity to the coast is strategic to ensure safety and to provide clear trajectories over water.

In [39]:
# Task 2: Mark the success/failed launches for each site on the map
# Add markers for each launch with success/failure indication
for index, launch in spacex_df.iterrows():
    coordinate = [launch['Lat'], launch['Long']]
    color = 'green' if launch['class'] == 1 else 'red'
    popup_text = f"{launch['Launch Site']} - {'Success' if launch['class'] == 1 else 'Failure'}"
    
    # Create a circle for each launch
    circle = folium.Circle(
        coordinate, 
        radius=100, 
        color=color, 
        fill=True
    ).add_child(folium.Popup(popup_text))
    
    # Add the circle to the map
    site_map.add_child(circle)

# Display the map
site_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [40]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [41]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [42]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Create a new column 'marker_color' in spacex_df
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Create a Folium map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize the MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers to the MarkerCluster
for index, launch in spacex_df.iterrows():
    coordinate = [launch['Lat'], launch['Long']]
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=launch['marker_color']),
        popup=f"{launch['Launch Site']} - {'Success' if launch['class'] == 1 else 'Failure'}"
    )
    marker.add_to(marker_cluster)

# Display the map
site_map

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [43]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # Create and add a Marker cluster to the site map
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f"{record['Launch Site']} - {'Success' if record['class'] == 1 else 'Failure'}",
        icon=folium.Icon(
            color='white',
            icon_color=record['marker_color'],
            icon='rocket',
            prefix='fa'
        )
    )
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [46]:
# TASK 3: Calculate the distances between a launch site to its proximities
import math
import pandas as pd

# Function to calculate distance using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# Function to find the closest city
def find_closest_city(lat, lon, cities_df):
    cities_df['distance'] = cities_df.apply(lambda row: calculate_distance(lat, lon, row['Lat'], row['Long']), axis=1)
    closest_city = cities_df.loc[cities_df['distance'].idxmin()]
    return closest_city['City'], closest_city['distance']

# TASK 3: Calculate the distances between a launch site to its proximities

# Get unique launch sites
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()

# NASA Johnson Space Center coordinates
nasa_coordinate = (29.559684888503615, -95.0830971930759)

# Calculate distances between launch sites
print("Distances between launch sites:")
for i, site1 in launch_sites.iterrows():
    for j, site2 in launch_sites.iterrows():
        if i < j:  # Avoid calculating the same pair twice
            distance = calculate_distance(site1['Lat'], site1['Long'], site2['Lat'], site2['Long'])
            print(f"{site1['Launch Site']} to {site2['Launch Site']}: {distance:.2f} km")

# Calculate distances from NASA Johnson Space Center
print("\nDistances from NASA Johnson Space Center:")
for _, site in launch_sites.iterrows():
    distance = calculate_distance(nasa_coordinate[0], nasa_coordinate[1], site['Lat'], site['Long'])
    print(f"{site['Launch Site']}: {distance:.2f} km")

# Find closest cities (assuming you have a cities_df DataFrame)
# If you don't have cities_df, comment out or remove this section
print("\nClosest cities to launch sites:")
for _, site in launch_sites.iterrows():
    closest_city, distance = find_closest_city(site['Lat'], site['Long'], cities_df)
    print(f"{site['Launch Site']}: {closest_city} ({distance:.2f} km)")

Distances between launch sites:
CCAFS LC-40 to VAFB SLC-4E: 3825.84 km
CCAFS LC-40 to KSC LC-39A: 6.90 km
CCAFS LC-40 to CCAFS SLC-40: 0.11 km
VAFB SLC-4E to KSC LC-39A: 3819.05 km
VAFB SLC-4E to CCAFS SLC-40: 3825.85 km
KSC LC-39A to CCAFS SLC-40: 6.93 km

Distances from NASA Johnson Space Center:
CCAFS LC-40: 1413.33 km
VAFB SLC-4E: 2462.75 km
KSC LC-39A: 1406.43 km
CCAFS SLC-40: 1413.37 km

Closest cities to launch sites:


<class 'NameError'>: name 'cities_df' is not defined

Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [47]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [48]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [50]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

import folium
from folium.plugins import MousePosition
import math

# Function to calculate distance using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# Assuming we have a launch site's coordinates
launch_site_lat = 28.5618571  # Example: Cape Canaveral
launch_site_lon = -80.577366

# Create a map centered on the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add a marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="red", icon="info-sign")
).add_to(site_map)

# Add MousePosition to the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position="topright",
    separator=" : ",
    empty_string="NaN",
    lng_first=True,
    num_digits=20,
    prefix="Coordinates:",
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Add instructions to the map
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Click on the map to mark the closest coastline point",
    icon=folium.Icon(color="green", icon="info-sign")
).add_to(site_map)

# JavaScript function to handle click events
site_map.add_child(folium.LatLngPopup())

# Display the map
site_map

In [51]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

import folium
from folium.features import DivIcon
import math

# Function to calculate distance using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# Assuming we have a launch site's coordinates
launch_site_lat = 28.5618571  # Example: Cape Canaveral
launch_site_lon = -80.577366

# Create a map centered on the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add a marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="red", icon="info-sign")
).add_to(site_map)

# Coordinates for the closest coastline point (you would determine these)
coastline_lat = 28.56367  # Example coordinate, replace with actual closest coastline point
coastline_lon = -80.57163

# Calculate the distance
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create and add a folium.Marker on your selected closest coastline point on the map
coastline_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    popup="Closest Coastline Point",
    icon=folium.Icon(color="blue", icon="info-sign")
)
coastline_marker.add_to(site_map)

# Display the distance between coastline point and launch site using the icon property
distance_marker = folium.Marker(
    [(launch_site_lat + coastline_lat) / 2, (launch_site_lon + coastline_lon) / 2],  # Midpoint
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(75, 18),
        html=f'<div style="font-size: 12pt; color:#d35400;"><b>{distance:.2f} KM</b></div>',
    )
)
distance_marker.add_to(site_map)

# Add a line connecting the launch site and coastline point
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color="green",
    weight=2,
    opacity=0.8
).add_to(site_map)

# Display the map
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [57]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [59]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
import folium
from folium.features import DivIcon
import math

# Function to calculate distance using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# Assuming we have a launch site's coordinates
launch_site_lat = 28.5618571  # Example: Cape Canaveral
launch_site_lon = -80.577366

# Create a map centered on the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add a marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="red", icon="info-sign")
).add_to(site_map)

# Coordinates for the closest city (or other point of interest)
# Replace these with actual coordinates of the closest city, railway, highway, etc.
closest_poi_lat = 28.3922  # Example: Cocoa Beach, FL
closest_poi_lon = -80.6077
poi_name = "Cocoa Beach"  # Replace with actual name

# Calculate the distance
distance = calculate_distance(launch_site_lat, launch_site_lon, closest_poi_lat, closest_poi_lon)

# Create a marker with distance to the closest point of interest
poi_marker = folium.Marker(
    [closest_poi_lat, closest_poi_lon],
    popup=f"{poi_name}: {distance:.2f} km from launch site",
    icon=folium.Icon(color="green", icon="info-sign")
)
poi_marker.add_to(site_map)

# Display the distance between the point of interest and launch site
distance_marker = folium.Marker(
    [(launch_site_lat + closest_poi_lat) / 2, (launch_site_lon + closest_poi_lon) / 2],  # Midpoint
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(75, 18),
        html=f'<div style="font-size: 12pt; color:#d35400;"><b>{distance:.2f} KM</b></div>',
    )
)
distance_marker.add_to(site_map)

# Draw a line between the marker and the launch site
coordinates = [
    [launch_site_lat, launch_site_lon],
    [closest_poi_lat, closest_poi_lon]
]
line = folium.PolyLine(locations=coordinates, weight=2, color='blue', opacity=0.8)
site_map.add_child(line)

# Display the map
site_map

Are launch sites in close proximity to railways?: No, I cannot find any railways close by. The distance from populated areas and critical infrastructure is a key factor in launch site placement. This explains why they are not too close to railways.

Are launch sites in close proximity to highways? No, I cannot find any highways close by. The distance from populated areas and critical infrastructure is a key factor in launch site placement. This explains why they are not too close to highways

Are launch sites in close proximity to coastline? Yes, it is located close to coastlines. The proximity to coastlines is a common feature of many launch sites. This allows for safer launch trajectories over water and easier access to various orbits

Do launch sites keep certain distance away from cities? Yes, it is far away from the cities. The distance from populated areas and critical infrastructure is a key factor in launch site placement. This explains why they are not too close to cities

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
